# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [61]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [62]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [63]:
import os
from glob import glob

# Write your code below
#glob(os.getenv('PRICE_DATA'), '**/*.parquet'))
PRICE_DATA = os.getenv('PRICE_DATA')
price_files = glob(os.path.join(PRICE_DATA, '**/*.parquet'), recursive=True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [82]:
import pandas as pd
# Write your code below.
#load parquet files into Dask DataFrame
dd_px = dd.read_parquet(price_files).set_index('ticker')

#create Close and Adj Close lag columns for each ticker
px_feat = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending=True)
                        .assign(Close_lag_1 = x['Close'].shift(1))
                        .assign(Adj_lag_1 = x['Adj Close'].shift(1)),
            meta = pd.DataFrame(data = {
                'Date': 'datetime64[ns]',
                'Open': 'f8',
                'High': 'f8',
                'Low': 'f8',
                'Close': 'f8',
                'Adj Close': 'f8',
                'Volume': 'i8',
                'source': 'object',
                'Year': 'int32',
                'Close_lag_1': 'f8',
                'Adj_lag_1': 'f8'
            }, 
            index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
        )
)

#augment Dask DataFrame with columns returns and hi_lo_range
px_feat['returns'] = px_feat['Close']/px_feat['Close_lag_1'] - 1
px_feat['hi_lo_range'] = px_feat['High'] - px_feat['Low']


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [83]:
# Write your code below.
#compute values in Dask DataFrame to return pandas DataFrame
px_feat_pd = px_feat.compute()


In [84]:
#calculate the 10 day rolling average of returns
px_feat_pd['returns_rolling_avg'] = px_feat_pd['returns'].rolling(10).mean()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It was not necessary to convert to pandas before calculating the moving average return - the same manipulations (i.e., the `rolling()` method) could have been performed on the Dask DataFrame.

It would have been better to do it in Dask for the following reasons:
+ Computing the rolling average from a pandas DataFrame requires all values to be computed upfront, regardless of whether they will all be accessed eventually. If it is done in Dask, a strict subset of computations may ultimately be sufficient.
+ When the DataFrame is computed first, the `returns` column must be created and stored in memory as `Close` is traversed. Then, when the rolling average is introduced, the `returns` column must be traversed in a separate operation. In contrast, had the rolling average column been added in Dask, it could have created both the `returns` and `returns_rolling_average` columns in the initial traversal of `Close`.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.